In [4]:
from time import time
import bitstring as bit

# initial, inverse, and E-bit permutation arrays
Init_Perm = [2, 6, 3, 1, 4, 8, 5, 7]
Inv_Perm = [4, 1, 3, 5, 7, 2, 8, 6]
E_P = [4, 1, 2, 3, 2, 3, 4, 1]

# Permutation arrays
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
P8 = [6, 3, 7, 4, 8, 5, 10, 9]
P4 = [2, 4, 3, 1]

#S boxes for substitutions
S = {
  "00": 0,
  "01": 1,
  "10": 2,
  "11": 3
}
S1 = [["01", "00", "11", "10"],
      ["11", "10", "01", "00"],
      ["00", "10", "01", "11"],
      ["11", "01", "11", "10"]]
S2 = [["00", "01", "10", "11"],
      ["10", "00", "01", "11"],
      ["11", "00", "01", "00"],
      ["10", "01", "00", "11"]]

def Subs(s0, s1):
    w = S[s0[0] + s0[3]]
    x = S[s0[1] + s0[2]]
    y = S[s1[0] + s1[3]]
    z = S[s1[1] + s1[2]]
    return S1[w][x] + S2[y][z]

# uses the length of the key divided by 2 to return the Left or right of the permuted key for C0 and D0 respectively
def Left(permedKey):
    return permedKey[:int((len(permedKey)/2))]

def Right(permedKey):
    return permedKey[int((len(permedKey)/2)):]

# Leftshift for key generation
def LeftShift(unshifted, shifts):
    if shifts == 1:
        return unshifted[1:] + unshifted[0]
    elif shifts == 2:
        return unshifted[2:] + unshifted[0] + unshifted[1]

#Permutation for SDES
def Perm(oldString, perm):
    newString = ''
    for i in perm:
        newString += oldString[i-1]
    return newString

def XOR_Part(bits, key):
    new = ''
    for bit, key_bit in zip(bits, key):
        new += str(((int(bit) + int(key_bit)) % 2))
    return new
    

if __name__ == "__main__":

    #----------------------------------- Key Generation Steps ----------------------------------------------

    Key = '0000000000'
    ChangedKey = ''

    # 10 bit key permutation
    ChangedKey = Perm(Key, P10)
    
    # splitting key into C_0 and D_0
    C_0 = Left(ChangedKey)
    D_0 = Right(ChangedKey)
    
    #First left shifts (1 left shift)
    C_1 = LeftShift(C_0, 1)
    D_1 = LeftShift(D_0, 1)
    
    # First Round Key
    Key1 = C_1 + D_1
    RK_1 = Perm(Key1, P8)

    # Second Left shift (2 left shifts)
    C_2 = LeftShift(C_1, 2)
    D_2 = LeftShift(D_1, 2)

    # Second Round Key
    Key2 = C_2 + D_2
    RK_2 = Perm(Key2, P8)

    # Third Left shift (2 left shifts)
    C_3 = LeftShift(C_2, 2)
    D_3 = LeftShift(D_2, 2)

    # Third Round Key
    Key3 = C_3 + D_3
    RK_3 = Perm(Key3, P8)

    # Fourth Left shift (2 left shifts)
    C_4 = LeftShift(C_3, 2)
    D_4 = LeftShift(D_3, 2)

    # Fourth Round Key
    Key4 = C_4 + D_4
    RK_4 = Perm(Key4, P8)
    
    # ------------------------------------ Actual SDES Rounds --------------------------------------------
    plaintext = '01000000'
    
    #Initial Permutation
    Alt_Mess = Perm(plaintext, Init_Perm)
    
    #split into sides
    L_0 = Left(Alt_Mess)
    R_0 = Right(Alt_Mess)
    
    # E-bit Selection
    Alt_R0 = Perm(R_0, E_P)
    
    #XOR step
    XOR_output = XOR_Part(Alt_R0, RK_1)
    
    #Substitution
    s0 = Left(XOR_output)
    s1 = Right(XOR_output)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again = XOR_Part(L_0, p4)
    
    # MESSAGE AFTER FIRST ROUND OF SDES
    Alt_Mess1 = R_0 + XOR_again
    
    #split into sides
    L_1 = R_0
    R_1 = XOR_again
    
    # E-bit Selection
    Alt_R1 = Perm(R_1, E_P)
    
    #XOR step
    XOR_output1 = XOR_Part(Alt_R1, RK_2)
    
    #Substitution
    s0 = Left(XOR_output)
    s1 = Right(XOR_output)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again1 = XOR_Part(L_1, p4)
    
    # MESSAGE AFTER SECOND ROUND OF SDES
    Alt_Mess2 = R_1 + XOR_again1
    
    #split into sides
    L_2 = R_1
    R_2 = XOR_again1
    
    # E-bit Selection
    Alt_R2 = Perm(R_2, E_P)
    
    #XOR step
    XOR_output2 = XOR_Part(Alt_R2, RK_3)
    
    #Substitution
    s0 = Left(XOR_output2)
    s1 = Right(XOR_output2)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again2 = XOR_Part(L_2, p4)
    
    # MESSAGE AFTER THIRD ROUND OF SDES
    Alt_Mess3 = R_2 + XOR_again2
    
    #split into sides
    L_3 = R_2
    R_3 = XOR_again2
    
    # E-bit Selection
    Alt_R3 = Perm(R_3, E_P)
    
    #XOR step
    XOR_output3 = XOR_Part(Alt_R3, RK_4)
    
    #Substitution
    s0 = Left(XOR_output3)
    s1 = Right(XOR_output3)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again3 = XOR_Part(L_3, p4)
    
    # MESSAGE AFTER FOURTH ROUND OF SDES
    Alt_Mess4 = R_3 + XOR_again3
    
    #split into sides
    L_4 = R_3
    R_4 = XOR_again3
    PreOutput = R_4 + L_4
    
    #Final Encypted plaintext
    ciphertext = Perm(PreOutput, Inv_Perm)
    print(ciphertext)

10111110
